In [1]:
# !wget https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz

--2018-10-24 20:03:36--  https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.99.189
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.99.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166373201 (159M) [application/x-tar]
Saving to: ‘yelp_review_polarity_csv.tgz’

yelp_review_polarit 100%[===================>] 158.67M  4.58MB/s    in 60s     

2018-10-24 20:04:36 (2.66 MB/s) - ‘yelp_review_polarity_csv.tgz’ saved [166373201/166373201]



In [3]:
# !tar -xvzf yelp_review_polarity_csv.tgz

yelp_review_polarity_csv/
yelp_review_polarity_csv/train.csv
yelp_review_polarity_csv/readme.txt
yelp_review_polarity_csv/test.csv


In [1]:
from fastai import *
from fastai.text import *

In [2]:
from fastai.core import *

In [3]:
path = Path('yelp_review_polarity_csv')

In [69]:
train_csv = path/'train.csv'

In [70]:
train = pd.read_csv(train_csv, header=None)

In [71]:
train.head()

,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [74]:
train.iloc[:,0] = train.iloc[:,0].apply(lambda x: 0 if x==1 else 1)

In [76]:
train.head()

,0,1
0,0,"Unfortunately, the frustration of being Dr. Go..."
1,1,Been going to Dr. Goldberg for over 10 years. ...
2,0,I don't know what Dr. Goldberg was like before...
3,0,I'm writing this review to give you a heads up...
4,1,All the food is great here. But the best thing...


In [77]:
train.to_csv(train_csv, header=None, index=None)

In [78]:
train = pd.read_csv(train_csv, header=None)

In [79]:
train.head()

,0,1
0,0,"Unfortunately, the frustration of being Dr. Go..."
1,1,Been going to Dr. Goldberg for over 10 years. ...
2,0,I don't know what Dr. Goldberg was like before...
3,0,I'm writing this review to give you a heads up...
4,1,All the food is great here. But the best thing...


In [75]:
train.iloc[0][1]

"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."

In [80]:
valid_csv = path/'valid.csv'

In [32]:
test_csv = path/'test.csv'

In [35]:
!mv $test_csv $valid_csv

In [81]:
valid = pd.read_csv(valid_csv, header=None)

In [82]:
valid.head()

,0,1
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


In [83]:
valid.iloc[:,0] = valid.iloc[:,0].apply(lambda x: 0 if x==1 else 1)

In [84]:
valid.to_csv(valid_csv, header=None, index=None)

In [21]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [25]:
!sudo pip install spacy

    100% |████████████████████████████████| 23.2MB 60kB/s 
    100% |████████████████████████████████| 81kB 12.7MB/s 
    100% |████████████████████████████████| 153kB 8.5MB/s 
    100% |████████████████████████████████| 614kB 2.3MB/s 
    100% |████████████████████████████████| 194kB 6.8MB/s 
    100% |████████████████████████████████| 1.9MB 808kB/s 
    100% |████████████████████████████████| 51kB 12.6MB/s 
    100% |████████████████████████████████| 296kB 4.4MB/s 
    100% |████████████████████████████████| 450kB 3.4MB/s 
  Running setup.py bdist_wheel for dill ... done
  Stored in directory: /root/.cache/pip/wheels/e2/5d/17/f87cb7751896ac629b435a8696f83ee75b11029f5d6f6bda72
  Running setup.py bdist_wheel for regex ... done
  Stored in directory: /root/.cache/pip/wheels/74/17/3f/c77bba99efd74ba1a19862c9dd97f4b6d735e2826721dc00ff
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1

In [26]:
!sudo python -m spacy download en

    100% |████████████████████████████████| 37.4MB 149.0MB/s 
  Running setup.py install for en-core-web-sm ... done

    Linking successful
    /usr/local/lib/python2.7/dist-packages/en_core_web_sm -->
    /usr/local/lib/python2.7/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [94]:
classes = read_classes(path/'classes.txt')
classes[0], classes[1]

('negative', 'positiv')

In [4]:
data_lm = TextLMDataBunch.from_csv(path)
data_clas = TextClasDataBunch.from_csv(path, vocab=data_lm.train_ds.vocab)

In [98]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.unfreeze()
learn.fit(2, slice(1e-4,1e-2))

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch-nightly_1540121100527/work/aten/src/THC/generic/THCTensorCopy.cpp:20

In [85]:
# data_lm = TextLMDataBunch.from_csv(path)

Tokenizing train.


Numericalizing train.
Tokenizing valid.


Numericalizing valid.


In [86]:
# data_clas = TextClasDataBunch.from_csv(path, vocab=data_lm.train_ds.vocab, bs=32)

In [40]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)

In [41]:
learn.fit_one_cycle(1, 1e-2)

Total time: 1:09:47
epoch  train loss  valid loss  accuracy
1      4.372054    4.162149    0.252220  (1:09:47)



In [42]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

Total time: 1:18:02
epoch  train loss  valid loss  accuracy
1      3.830066    3.688198    0.309435  (1:18:02)



In [43]:
learn.save_encoder('ft_enc')

In [87]:
data_clas.train_ds.classes

array([0, 1])

In [88]:
data_clas.train_ds.label_cols

[0]

In [89]:
data_clas.valid_ds.classes

array([0, 1])

In [90]:
data_clas.valid_ds.vocab_size

60002

In [91]:
data_clas.train_ds.vocab_size

60002

In [92]:
data_clas.valid_ds.label_cols

[0]

In [93]:
learn = RNNLearner.classifier(data_clas)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch-nightly_1540121100527/work/aten/src/THC/generic/THCTensorCopy.cpp:20

In [5]:
learn = RNNLearner.classifier(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)

Total time: 23:01
epoch  train loss  valid loss  accuracy
1      0.191463    0.131722    0.949263  (23:01)



In [6]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

Total time: 27:55
epoch  train loss  valid loss  accuracy
1      0.131822    0.082398    0.968026  (27:55)



In [7]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

Total time: 56:31
epoch  train loss  valid loss  accuracy
1      0.095156    0.071463    0.972789  (56:31)



In [8]:
learn.save_encoder('fine_tuned')